In [1]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [14]:
# Generic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import signal
from scipy.fft import fftshift
import scipy
from datetime import datetime

#ML Libs
from sklearn.model_selection import train_test_split, KFold
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from keras import backend as K
import scipy.ndimage as nd
from keras.models import Sequential,load_model
from keras import regularizers
from keras import initializers
from keras import metrics
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, BatchNormalization, ReLU, LeakyReLU, Conv2D
from keras import layers
from keras.optimizers import Adam, Adagrad, RMSprop
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, auc, roc_curve
from keras.layers.experimental import preprocessing


Terminal = False

#Loading SQI Matched with clinitcal to the Dataframe

In [3]:
#Loading SQI Matched with clinical to the Dataframe

id = '1DDf_vluJE-zg--41A0zpP5_9-FU-wsJp' # The shareable link id
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Complete_SQIs_with_Clinical.csv')  
SQI_C = pd.read_csv('Complete_SQIs_with_Clinical.csv')


id = '1zEpiX0yXT16cz3znpo8wjoV_U_8vSoyb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Raw_signals.csv')  
Raw = pd.read_csv('Raw_signals.csv')

if Terminal:
    print("\n SQI with Clinical Match:")
    print(SQI_C)
    print("\n Raw Signals")
    print(Raw)


#Not really need, here for completeness
event = ['event_shock', 'reshock24','diagnosis_admission',\
     'ascites', 'respiratory_distress', 'ventilation_cannula', \
     'ventilation_mechanical', 'ventilation_ncpap', 'bleeding_severe', \
     'cns_abnormal', 'liver_mild', 'pleural_effusion', 'skidney']

event_shock = 'shock_admission'

SQI_C['keep'] = False
for i in range(len(event)):
    event_s = event[i]
    SQI_C['keep'][SQI_C[event_s] == True] = True
    #print("\n Total ", event[i], " events:")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is tryi

In [38]:
#Patient 1 had no shock
#Patient 2 had shock 
P1times = datetime.strptime('20072020143000000', '%d%m%Y%H%M%S%f')
P1timef = datetime.strptime('20072020163000000', '%d%m%Y%H%M%S%f')

P2times = datetime.strptime('03112020103000000', '%d%m%Y%H%M%S%f')
P2timef = datetime.strptime('03112020123000020', '%d%m%Y%H%M%S%f') #added 20 miliseconds to match the length of Patient 1

P3times = datetime.strptime('18092020203000000', '%d%m%Y%H%M%S%f')##########
P3timef = datetime.strptime('18092020213000000', '%d%m%Y%H%M%S%f') 


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

Patient1 = Raw[(Raw.study_no == '003-2162') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2028') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]
Patient3 = Raw[(Raw.study_no == '003-2104') & (P3times <= Raw.PPG_Datetime) & (P3timef > Raw.PPG_Datetime)]#############


Patient1['Label'] = 0 
Patient2['Label'] = 1
Patient3['Label'] = 1

if Terminal:
  print(Patient1)
  print(Patient2)
  print(Patient3)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 100)
Patient2_w = np.array_split(Patient2, 100)
Patient3_w = np.array_split(Patient3, 50)##########

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient1_w),7200,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'IR_ADC_bpf') #Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'IR_ADC_bpf')
Patient3_w_arr = choose_signal(Patient3_w, 'IR_ADC_bpf')#############


Patient1_w_arr_l = np.zeros((100,1), dtype=int)
Patient2_w_arr_l = np.ones((100,1), dtype=int)
Patient3_w_arr_l = np.ones((100,1), dtype=int)############

#Defining Sampling rate
fs = 100

Score_array = np.empty((5,1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [41]:
##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr,Patient2_w_arr))
Labels = np.vstack((Patient1_w_arr_l,Patient2_w_arr_l))

counter = 0

k = KFold(n_splits=5, random_state=None, shuffle=True)
for train_index, test_index in k.split(Data):
  X_train = Data[train_index]
  X_test = Data[test_index]
  y_train = Labels[train_index]
  y_test = Labels[test_index]

  if Terminal:
    print(train_index)
    print(test_index)
  #Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

  #Training_sets = np.vstack((P1_tr,P2_tr))
  Globalmean = np.mean(X_train)
  Globalstd = np.std(X_train)


  P_train = (X_train - Globalmean) / Globalstd
  P_test = (X_test - Globalmean) / Globalstd



  if Terminal:
    print(P1_training.shape)
    print(P1_test.shape)

  #Empty arrays
  #P1Sxx = np.empty((len(P1_training),26,32))
  #P1Sxxtest = np.empty((len(P1_test),26,32))
  PSxx = np.empty((len(P_train),26,32))
  PSxxtest = np.empty((len(P_test),26,32))


  #Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
  #Patient 1

  def spectrogram_arr(Input, Output_arr, fs):
    for i in range(len(Input)):
      f, t, Sxx = signal.spectrogram(Input[i], fs)
      fmin = 0 # Hz
      fmax = 10 # Hz
      freq_slice = np.where((f >= fmin) & (f <= fmax))
      # keep only frequencies of interest
      f   = f[freq_slice]
      Sxx = Sxx[freq_slice,:][0]
      Output_arr[i] = Sxx
    return Output_arr

  X_train = spectrogram_arr(P_train, PSxx, fs)
  X_test = spectrogram_arr(P_test, PSxxtest, fs)


  if Terminal:
    print("\n P1 Training array Shape")
    print(P1Sxx.shape)
    print("\n P1 Test array Shape")
    print(P1Sxxtest.shape)
    print("\n P2 Training array Shape")
    print(P2Sxx.shape)
    print("\n P2 Test array Shape")
    print(P2Sxxtest.shape)

  if Terminal:
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    print(ZPred.shape)
    print(Patient3_w_arr_l.shape)


  #MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
  #tf.random.set_seed(1234)

  #converting to categorical
  y_trainf = tf.keras.utils.to_categorical(y_train)
  y_testf = tf.keras.utils.to_categorical(y_test)
  X_trainf = X_train.reshape(160, 26, 32, 1)
  X_testf = X_test.reshape(40, 26, 32, 1)

  if Terminal:
    print('Train Labels')
    print(y_trainf.shape)
    print('Train Data')
    print(X_trainf.shape)

  #Best IR_ADC MODEL===========================lr 3e-3====================================


  data_augmentation = Sequential(
      [
          preprocessing.RandomZoom(0.2)

      ]
  )


  initialization = 'glorot_uniform'
  model = Sequential()
  model.add(data_augmentation)
  model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer=initialization))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
  model.add(Dropout(0.4))
  model.add(Conv2D(64, (2, 2), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
  model.add(Conv2D(128, (2, 2), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
  model.add(GlobalAveragePooling2D())
  model.add(Dense(64,'relu'))
  model.add(Dense(32,'relu'))
  model.add(Dense(16,'relu'))
  model.add(Dense(2))
  model.add(Activation('softmax'))

  # . . . 

  #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=200, restore_best_weights=True)

  model.compile(optimizer=Adam(learning_rate=3e-3),loss='categorical_crossentropy', metrics=['categorical_accuracy'])

  history = model.fit(X_trainf, y_trainf, epochs=350, validation_split=0.2)#, callbacks=[early_stopping])

  score = model.evaluate(X_testf, y_testf)

  Score_array[counter] = score[1]
  counter = counter + 1

print(Score_array)

print("\n K-Fold CV Max Test Accuracy:")
print(np.max(Score_array))

print("\n K-Fold CV Min Test Accuracy:")
print(np.min(Score_array))

print("\n K-Fold CV Average Test Accuracy:")
print(np.sum(Score_array)/5)

Epoch 1/350
4/4 [==============================] - 3s 169ms/step - loss: 0.7165 - categorical_accuracy: 0.5698 - val_loss: 0.7736 - val_categorical_accuracy: 0.0000e+00
Epoch 2/350
4/4 [==============================] - 0s 10ms/step - loss: 0.6889 - categorical_accuracy: 0.6146 - val_loss: 0.7691 - val_categorical_accuracy: 0.0000e+00
Epoch 3/350
4/4 [==============================] - 0s 10ms/step - loss: 0.6373 - categorical_accuracy: 0.5823 - val_loss: 0.7796 - val_categorical_accuracy: 0.0000e+00
Epoch 4/350
4/4 [==============================] - 0s 10ms/step - loss: 0.6159 - categorical_accuracy: 0.6115 - val_loss: 0.7486 - val_categorical_accuracy: 0.0000e+00
Epoch 5/350
4/4 [==============================] - 0s 10ms/step - loss: 0.6285 - categorical_accuracy: 0.5844 - val_loss: 0.7001 - val_categorical_accuracy: 0.5000
Epoch 6/350
4/4 [==============================] - 0s 10ms/step - loss: 0.6214 - categorical_accuracy: 0.6719 - val_loss: 0.6891 - val_categorical_accuracy: 0.6875